In [1]:
import numpy as np
from scipy.spatial import distance_matrix

# Algorithm implementation

In [107]:
# inspired by neurokit2.complexity.complexity_lyapunov

def z_vector_embedding(signal, delay=1, dimension=3):
    n = len(signal)
    z_len = n - (dimension - 1) * delay

    z = np.zeros((dimension, z_len))
    for i in range(dimension):
        z[i] = signal[(i * delay):(i * delay + z_len)]
    return z.T


def largest_lyapunov_exponent(signal, delay=1, dimension=4, separation=1, len_trajectory=20):
    embedded = z_vector_embedding(signal, delay=delay, dimension=dimension)
    n = len(embedded)

    distances = distance_matrix(embedded, embedded)
    # remove too close points from consideration
    for i in range(n):
        distances[i, max(0, i-separation):(i + separation + 1)] = np.inf

    max_trajectory_start = n - len_trajectory + 1
    neighbor_indices = np.argmin(distances[:max_trajectory_start, :max_trajectory_start], axis=1)

    trajectories = []
    x = np.arange(max_trajectory_start)  # trajectories to track
    x_neighbor = neighbor_indices  # their neighbors
    for _ in range(len_trajectory):
        divergence = distances[x, x_neighbor]
        mean_log_divergence = np.mean(np.log(divergence))
        trajectories.append(mean_log_divergence)
        # advance time one step
        x += 1
        x_neighbor += 1

    times = np.arange(len_trajectory) + 1
    slope, _ = np.polyfit(times, trajectories, deg=1)
    return slope

# Data

In [124]:
from series import skew_tent_map, logistic_map, henon_map, schuster_map, henon_map, lorenz_map

In [125]:
sine_data = np.sin(np.arange(0, 100, .01))
gaussian_noise = np.random.normal(size=1000)

In [126]:
from statsmodels.tsa.arima_process import arma_generate_sample
ar = np.array([1, 0.75, -0.25])
ma = np.array([1, 0.65, 0.35])
arma_process = arma_generate_sample(ar, ma, 1000)

In [127]:
import yfinance as yf
snp = yf.download("^GSPC", start='1993-02-01', end='2023-12-17')
snp_return = np.diff(snp["Close"])

[*********************100%%**********************]  1 of 1 completed


In [128]:
series = {
    "Gaussian": gaussian_noise,
    "ARMA": arma_process,
    "Sine": sine_data,
    "Financial": snp_return,
    "Lorenz series": lorenz_map(1000),
    "Skew Tent map": skew_tent_map(1000),
    "Logistic map": logistic_map(1000),
    "Schuster map": schuster_map(1000),
    "Henon map": henon_map(1000)
}

# Estimation

In [133]:
for k, v in series.items():
    lle = largest_lyapunov_exponent(v, delay=1, dimension=6, separation=100)
    print(k, "LLE:", round(lle, 4))

Gaussian LLE: 0.0394
ARMA LLE: 0.0531
Sine LLE: -0.0
Financial LLE: 0.0449
Lorenz series LLE: 0.0888
Skew Tent map LLE: 0.1988
Logistic map LLE: 0.1039
Schuster map LLE: 0.1522
Henon map LLE: 0.182
